In [152]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px

from itertools import product


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


In [186]:
from collections import OrderedDict

# Data

######## 2017 ##########

# Présidentielle
candidats_pres_2017 = OrderedDict({
    "DUPONT-AIGNAN": {'id': 0, 'nom': 'DUPONT-AIGNAN Nicolas'} ,
    "LE PEN": {'id': 1, 'nom': 'DUPONT-AIGNAN Nicolas'} ,
    "MACRON": {'id': 2, 'nom': 'MACRON Emmanuel'} ,
    "HAMON": {'id': 3, 'nom': 'HAMON Benoît'} ,
    "ARTHAUD": {'id': 4, 'nom': 'ARTHAUD Nathalie'} ,
    "POUTOU": {'id': 5, 'nom': 'POUTOU Philippe'} ,
    "CHEMINADE": {'id': 6, 'nom': 'CHEMINADE Jacques'} ,
    "LASSALLE": {'id': 7, 'nom': 'LASSALLE Jean'} ,
    "MÉLENCHON": {'id': 8, 'nom': 'MÉLENCHON Jean-Luc'} ,
    "ASSELINEAU" : {'id': 9, 'nom': 'ASSELINEAU François'} ,
    "FILLON": {'id': 10, 'nom': 'FILLON François'} 
})

groupes_politiques_pres_2017 = {
    'macron': ['MACRON'],
    'droite': ['FILLON'],
    'gauche': ['MÉLENCHON', 'HAMON', 'ARTHAUD', 'POUTOU'],
    'extreme_droite': ['DUPONT-AIGNAN', 'LE PEN'],
    'autre': ['ASSELINEAU', 'LASALLE', 'CHEMINADE']
}

# Législatives
candidats_leg_2017 = OrderedDict({
    "PIERINI": {'id': 0, 'nom': "PIERINI Lucien"} ,
    "CHOTARD": {'id': 1, 'nom': 'CHOTARD Stéphane'} ,
    "REYNARD": {'id': 2, 'nom': 'REYNARD Luc'} ,
    "LECELLIER": {'id': 3, 'nom': 'LECELLIER Cédric'} ,
    "DAURES": {'id': 4, 'nom': 'DAURES Anne'} ,
    "KADLER": {'id': 5, 'nom': 'KADLER Marie-Christine'} , 
    "MOREL": {'id': 6, 'nom': 'MOREL Véronique'} ,
    "VIARD": {'id': 7, 'nom': 'VIARD Jean'} ,
    "AUBERT": {'id': 8, 'nom': 'AUBERT Julien'} ,
    "CHALENÇON": {'id': 9, 'nom': 'CHALENÇON Christophe'} ,
    "KRAFT": {'id': 10, 'nom': 'KRAFT Sandrine'} ,
    "FOREST": {'id': 11, 'nom': 'FOREST Nathalie'} ,
    "THOMAS DE MALEVILLE": {'id': 12, 'nom': 'THOMAS DE MALEVILLE Marie'} 
})


groupes_politiques_leg_2017 = {
    'macron': ['VIARD'],
    'droite': ['AUBERT', 'KRAFT'],
    'gauche': ['CHOTARD', 'DAURES', 'REYNARD', 'KADLER'],
    'extreme_droite': ['THOMAS DE MALEVILLE'],
    'autre': ['CHALENÇON', 'MOREL', 'LECELLIER', 'PIERINI']
}

########### 2022 ############
# Présidentielle
groupes_politiques_pres_2022 = {
		'non_candidat': ['abstention', 'blancs', 'nuls'],

		'macron': ['MACRON', 'PÉCRESSE'],
		'gauche': ['MÉLENCHON', 'JADOT', 'ROUSSEL', 'HIDALGO', 'POUTOU', 'ARTHAUD'],

		'droite': ['LE PEN', 'ZEMMOUR', 'DUPONT-AIGNAN'],
		'autre': ['LASSALLE']
}

candidats_2022 = {
	"ARTHAUD": {'id': 1, 'nom': 'ARTHAUD Nathalie'} ,
	"ROUSSEL": {'id': 2, 'nom': 'ROUSSEL Fabien'} ,
	"ZEMMOUR": {'id': 3, 'nom': 'ZEMMOUR Éric'} ,
	"HIDALGO": {'id': 4, 'nom': 'HIDALGO Anne'} ,
	"JADOT": {'id': 5, 'nom': 'JADOT Yannick'} ,
	"PÉCRESSE": {'id': 6, 'nom': 'PÉCRESSE Valérie'} ,
	"MACRON": {'id': 7, 'nom': 'MACRON Emmanuel'} ,
    "LASSALLE": {'id': 8, 'nom': 'LASSALLE Jean'} ,
    "LE PEN": {'id': 9, 'nom': 'DUPONT-AIGNAN Nicolas'} ,
    "MÉLENCHON": {'id': 10, 'nom': 'MÉLENCHON Jean-Luc'} ,
    "POUTOU": {'id': 11, 'nom': 'POUTOU Philippe'} ,
    "DUPONT-AIGNAN": {'id': 12, 'nom': 'DUPONT-AIGNAN Nicolas'} ,
}

In [234]:
class VoteResults:
    def __init__(self, data, candidats, groupes_politiques, election="présidentielle"):
        self.data = data.copy()
        self.candidats = list(candidats.keys())
        self.election = election
        
        self.cols_bureau = ["code_dpt",
            "dpt",
            "code_circ",
            "circ",
            "code_commune",
            "commune",
            "code_b_vote"]

        self.cols_transform = ["cp", "id_b_vote"]

        self.cols_bureau_stats = [
            "inscrits",
            "abstentions",
            "p_abs/ins",
            "votants",
            "p_vot/ins",
            "blancs",
            "p_blancs/ins",
            "p_blancs/vot",
            "nuls",
            "p_nuls/ins",
            "p_nuls/vot",
            "exprimés", 
            "p_exp/ins",
            "p_exp/vot"]

        if (self.election == "présidentielle"):
            self.cols_candidats = ["n_panneau","sexe", "nom", "prenom", "voix", "p_voix/ins", "p_voix/exp"]
        else:
            self.cols_candidats = ["n_panneau","sexe", "nom", "prenom", "nuance", "voix", "p_voix/ins", "p_voix/exp"]
        
        self.data_format = self.parse_data_gouv()
        #print(self.data_format.head())
        self.circ_5 = self.parse_circ_5()
        #print(self.circ_5.head())
        self.bureaux = self.parse_bureaux()
        self.resultats_complets = self.parse_resultats_complet()
    
    def parse_data_gouv(self):
        

        data_format = self.data.copy()
        self.all_candidates_columns = list(map(lambda x: x[0] + '_' + x[1], product(self.candidats, self.cols_candidats)))
        data_format = data_format.iloc[:, 0: len(self.cols_bureau + self.cols_bureau_stats + self.all_candidates_columns)]
        
        data_format.columns = self.cols_bureau + self.cols_bureau_stats + self.all_candidates_columns

        data_format['id_b_vote'] = data_format.commune.astype(str) + '-' + data_format.code_b_vote.astype(str)
        data_format['cp'] = data_format.code_dpt.astype(str) + list(map(lambda s: str(s).rjust(3, '0'), data_format.code_commune))

        return data_format[ self.cols_bureau + self.cols_transform + self.cols_bureau_stats  + list(map(lambda x: x[0] + '_' + x[1], product(self.candidats, self.cols_candidats)))  ]

    def parse_circ_5(self):
        # Les bureaux
        return self.data_format[
            (self.data_format.code_dpt == 84) & 
            (self.data_format.code_circ == 5)
        ]

    def parse_bureaux(self):
        return self.circ_5.loc[:, self.cols_bureau + self.cols_transform + self.cols_bureau_stats ]

    def parse_resultats_complet(self):
        resultats_candidats = self.circ_5.loc[:, self.all_candidates_columns]

        res = []
        for i in range(len(self.candidats)):
            col_candidate_i = list(map(lambda x: x[0] + '_' + x[1], product([self.candidats[i]], self.cols_candidats) ))
            data_candidate = resultats_candidats.loc[ :, col_candidate_i ]
            data_candidate.columns = self.cols_candidats

            #res.append(data_candidate)
            res.append(pd.concat([self.bureaux, data_candidate], axis=1))

        resultats_candidats = pd.concat(res, axis=0)

        autres_votes = pd.DataFrame()

        for line in self.bureaux.iterrows():
            item = line[1]
            if (self.election == "présidentielle"):
                vote_bureau = pd.DataFrame({
                        'n_panneau': ['-', '-', '-'],
                        'sexe': ['-', '-', '-'],
                        'nom': ['abstention', 'blancs', 'nuls'],
                        'prenom': ['-', '-', '-'],
                        'voix': [item['abstentions'], item['blancs'], item['nuls'] ] ,
                        'p_voix/ins': [ item['p_abs/ins'], item['p_blancs/ins'], item['p_nuls/ins'] ],
                        'p_voix/exp': [ 0, item['p_blancs/vot'], item['p_nuls/vot'] ]
                    }) 
            else:
                vote_bureau = pd.DataFrame({
                    'n_panneau': ['-', '-', '-'],
                    'sexe': ['-', '-', '-'],
                    'nom': ['abstention', 'blancs', 'nuls'],
                    'prenom': ['-', '-', '-'],
                    'nuance': ['-', '-', '-'],
                    'voix': [item['abstentions'], item['blancs'], item['nuls'] ] ,
                    'p_voix/ins': [ item['p_abs/ins'], item['p_blancs/ins'], item['p_nuls/ins'] ],
                    'p_voix/exp': [ 0, item['p_blancs/vot'], item['p_nuls/vot'] ]
                }) 

            for col in self.cols_bureau + self.cols_transform + self.cols_bureau_stats:
                vote_bureau[col] = item[col]

            autres_votes = pd.concat([autres_votes, vote_bureau ], axis=0)
        autres_votes = autres_votes[ self.cols_bureau + self.cols_transform + self.cols_bureau_stats + self.cols_candidats]

        res_complets = pd.concat([autres_votes, resultats_candidats], axis=0)
        return res_complets

# Présidentielle

## 2022

In [235]:
pres_2022 = pd.read_csv("./raw/pres_2022_BV.csv",
    sep=";", 
    header=0)

presidentielle2022 = VoteResults(pres_2022, candidats_2022, groupes_politiques_pres_2022, 'présidentielle')
presidentielle2022.resultats_complets.to_csv('./transform/2022_pres_84_5_BV.csv', index=False)
presidentielle2022.resultats_complets.head()

/var/folders/cl/m20zwsz15vb_c47wccs1ghz40000gn/T/ipykernel_41907/279982418.py:1: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  pres_2022 = pd.read_csv("./raw/pres_2022_BV.csv",


,code_dpt,dpt,code_circ,circ,code_commune,commune,code_b_vote,cp,id_b_vote,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,n_panneau,sexe,nom,prenom,voix,p_voix/ins,p_voix/exp
0,84,Vaucluse,5,5ème circonscription,2,Ansouis,0001,84002,Ansouis-0001,840,166,19.76,674,80.24,14,1.67,2.08,2,0.24,0.30,658,78.33,97.63,-,-,abstention,-,166,19.76,0.00
1,84,Vaucluse,5,5ème circonscription,2,Ansouis,0001,84002,Ansouis-0001,840,166,19.76,674,80.24,14,1.67,2.08,2,0.24,0.30,658,78.33,97.63,-,-,blancs,-,14,1.67,2.08
2,84,Vaucluse,5,5ème circonscription,2,Ansouis,0001,84002,Ansouis-0001,840,166,19.76,674,80.24,14,1.67,2.08,2,0.24,0.30,658,78.33,97.63,-,-,nuls,-,2,0.24,0.30
0,84,Vaucluse,5,5ème circonscription,3,Apt,0001,84003,Apt-0001,972,268,27.57,704,72.43,10,1.03,1.42,5,0.51,0.71,689,70.88,97.87,-,-,abstention,-,268,27.57,0.00
1,84,Vaucluse,5,5ème circonscription,3,Apt,0001,84003,Apt-0001,972,268,27.57,704,72.43,10,1.03,1.42,5,0.51,0.71,689,70.88,97.87,-,-,blancs,-,10,1.03,1.42


## 2017

In [221]:
pres_2017 = pd.read_csv('./raw/pres_2017_BV.csv')

presidentielle2017 = VoteResults(pres_2017, candidats_pres_2017, groupes_politiques_pres_2017, 'présidentielle')
presidentielle2017.resultats_complets.head()

/var/folders/cl/m20zwsz15vb_c47wccs1ghz40000gn/T/ipykernel_41907/1423742485.py:1: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  pres_2017 = pd.read_csv('./raw/pres_2017_BV.csv')


,code_dpt,dpt,code_circ,circ,code_commune,commune,code_b_vote,cp,id_b_vote,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,n_panneau,sexe,nom,prenom,voix,p_voix/ins,p_voix/exp
0,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,84002,Ansouis-1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,-,-,abstention,-,142,16.15,0.00
1,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,84002,Ansouis-1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,-,-,blancs,-,24,2.73,3.26
2,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,84002,Ansouis-1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,-,-,nuls,-,6,0.68,0.81
0,84,Vaucluse,5,5ème circonscription,3,Apt,1,84003,Apt-1,923,189,20.48,734,79.52,10,1.08,1.36,7,0.76,0.95,717,77.68,97.68,-,-,abstention,-,189,20.48,0.00
1,84,Vaucluse,5,5ème circonscription,3,Apt,1,84003,Apt-1,923,189,20.48,734,79.52,10,1.08,1.36,7,0.76,0.95,717,77.68,97.68,-,-,blancs,-,10,1.08,1.36


In [225]:
presidentielle2017.resultats_complets.to_csv('./transform/2017_pres_84_5_BV.csv', index=False)

# Legislatives 2017

In [238]:
import pandas as pd
leg_2017 = pd.read_csv('./raw/leg_2017_BV.csv')

legislatives2017 = VoteResults(leg_2017, candidats_leg_2017, groupes_politiques_leg_2017, election="législatives")
legislatives2017.resultats_complets.tail()

/var/folders/cl/m20zwsz15vb_c47wccs1ghz40000gn/T/ipykernel_41907/2890291526.py:2: DtypeWarning: Columns (0,6,150,151,152,153,158,159,160,161,166,167,168,169,174,175,176,177,182,183,184,185,190,191,192,193,198,199,200,201,206,207,208,209,214,215,216,217,222,223,224,225,230,231,232,233) have mixed types. Specify dtype option on import or set low_memory=False.
  leg_2017 = pd.read_csv('./raw/leg_2017_BV.csv')


,code_dpt,dpt,code_circ,circ,code_commune,commune,code_b_vote,cp,id_b_vote,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,n_panneau,sexe,nom,prenom,nuance,voix,p_voix/ins,p_voix/exp
59376,84,Vaucluse,5,5ème circonscription,133,La Tour-d'Aigues,3,84133,La Tour-d'Aigues-3,1068,492,46.07,576,53.93,13,1.22,2.26,2,0.19,0.35,561,52.53,97.40,3.0,F,THOMAS DE MALEVILLE,Marie,FN,76.0,7.12,13.55
59411,84,Vaucluse,5,5ème circonscription,144,Viens,1,84144,Viens-1,482,194,40.25,288,59.75,6,1.24,2.08,1,0.21,0.35,281,58.30,97.57,3.0,F,THOMAS DE MALEVILLE,Marie,FN,48.0,9.96,17.08
59412,84,Vaucluse,5,5ème circonscription,145,Villars,1,84145,Villars-1,595,243,40.84,352,59.16,8,1.34,2.27,0,0.00,0.00,344,57.82,97.73,3.0,F,THOMAS DE MALEVILLE,Marie,FN,86.0,14.45,25.00
59417,84,Vaucluse,5,5ème circonscription,148,Villes-sur-Auzon,1,84148,Villes-sur-Auzon-1,972,454,46.71,518,53.29,8,0.82,1.54,5,0.51,0.97,505,51.95,97.49,3.0,F,THOMAS DE MALEVILLE,Marie,FN,78.0,8.02,15.45
59421,84,Vaucluse,5,5ème circonscription,151,Vitrolles-en-Lubéron,1,84151,Vitrolles-en-Lubéron-1,171,70,40.94,101,59.06,4,2.34,3.96,0,0.00,0.00,97,56.73,96.04,3.0,F,THOMAS DE MALEVILLE,Marie,FN,11.0,6.43,11.34


In [240]:
legislatives2017.resultats_complets.sort_values('id_b_vote').head()

KeyError: 'id-b-vote'

In [239]:
legislatives2017.resultats_complets.to_csv('./transform/2017_leg_84_5_BV.csv', index=False)

# Analyses

In [183]:
print('%s bureaux de votes, %s communes' % (len(circ_5), len(np.unique(resultats_bureaux.commune))))

112 bureaux de votes, 56 communes


In [187]:
np.unique(resultats_bureaux.commune)

array(['Ansouis', 'Apt', 'Aubignan', 'Aurel', 'Auribeau', 'Beaumettes',
       'Beaumont-de-Pertuis', 'Bedoin', 'Blauvac', "Cabrières-d'Aigues",
       'Caromb', 'Carpentras', 'Caseneuve', 'Castellet-en-Luberon',
       'Crillon-le-Brave', 'Flassan', 'Gargas', 'Gignac', 'Gordes',
       'Goult', 'Grambois', 'Joucas', 'La Bastide-des-Jourdans',
       'La Bastidonne', "La Motte-d'Aigues", "La Tour-d'Aigues",
       "Lagarde-d'Apt", 'Lioux', 'Loriol-du-Comtat', 'Malemort-du-Comtat',
       'Mirabeau', 'Modène', 'Monieux', 'Mormoiron', 'Murs', 'Méthamis',
       'Pertuis', "Peypin-d'Aigues", 'Roussillon', 'Rustrel', 'Saignon',
       'Saint-Christol', 'Saint-Hippolyte-le-Graveyron',
       'Saint-Martin-de-Castillon', 'Saint-Martin-de-la-Brasque',
       'Saint-Pantaléon', 'Saint-Pierre-de-Vassols',
       'Saint-Saturnin-lès-Apt', 'Saint-Trinit', 'Sannes', 'Sarrians',
       'Sault', 'Viens', 'Villars', 'Villes-sur-Auzon',
       'Vitrolles-en-Lubéron'], dtype=object)

In [191]:
fig = px.histogram(resultats_bureaux, y="commune", x="p_abs/ins", color="id_b_vote")
fig.show()

In [211]:
fig = px.box(resultats_bureaux.sort_values('p_abs/ins', ascending=False), 
    x="commune", 
    y="p_abs/ins",
    points="all",
    hover_data=resultats_bureaux.columns)
fig.show()

In [209]:
fig = px.box(resultats_bureaux.sort_values('abstentions', ascending=False), 
    x="commune", 
    y="abstentions",
    points="all",
    hover_data=resultats_bureaux.columns)
fig.show()

In [215]:
fig = px.box(resultats_bureaux.sort_values('p_abs/ins', ascending=False), 
    x="commune", 
    y="inscrits",
    points="all",
    hover_data=resultats_bureaux.columns)
fig.show()

In [196]:
resultats_bureaux[resultats_bureaux.commune == "Sault"]

,code_dpt,dpt,code_circ,circ,code_commune,commune,code_b_vote,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,id_b_vote
59624,84,Vaucluse,5,5ème circonscription,123,Sault,0001,1024,211,20.61,813,79.39,10,0.98,1.23,8,0.78,0.98,795,77.64,97.79,Sault-0001
59625,84,Vaucluse,5,5ème circonscription,123,Sault,0002,116,19,16.38,97,83.62,2,1.72,2.06,1,0.86,1.03,94,81.03,96.91,Sault-0002


In [181]:

#fig = px.histogram(res_circ_5, y="id_b_vote", x="voix", color="nom")
fig = px.histogram(resultats_bureaux, y='id_b_vote', x='p_abs/ins')
fig.show()

In [3]:
geo_bureaux = pd.read_csv("./geo_bureaux_de_vote.csv", header=0)
geo_bureaux.head()

,commune_code,libelle,code,libelle.1,num_voie,voie,complement1,complement2,lieu_dit,code_postal,...,circonscription_code,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_ligne,geo_l4,geo_l5
0,59350,Lille,807,ECOLE AMPERE,64,RUE D'IENA,NaN,NaN,NaN,59000,...,59-11,3.043027,50.621124,0.96,street,Rue d'Iena 59000 Lille,59350_4658,G,RUE D IENA,NaN
1,63058,Bulhon,1,BUREAU 1,NaN,Mairie,NaN,NaN,NaN,63350,...,63-05,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
2,97401,Les Avirons,001,MAIRIE DES AVIRONS,61,AV. DU GENERAL DE GAULLE,NaN,NaN,NaN,97425,...,974-07,55.333200,-21.242358,0.88,housenumber,61 Avenue du Général de Gaulle 97425 Les Avirons,97401_0127_00061,G,61 AVENUE DU GENERAL DE GAULLE,NaN
3,98712,Arue,1,Bureau 1,NaN,Ecole Ahutoru primaire,NaN,NaN,Ecole primaire Arue 1,98701,...,987-01,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN
4,54141,Coyviller,1,MAIRIE,NaN,NaN,NaN,NaN,NaN,NaN,...,54-04,NaN,NaN,0.00,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
geo_bureaux[geo_bureaux.circonscription_code == "85-05"]

,commune_code,libelle,code,libelle.1,num_voie,voie,complement1,complement2,lieu_dit,code_postal,...,circonscription_code,longitude,latitude,geo_score,geo_type,geo_adresse,geo_id,geo_ligne,geo_l4,geo_l5
119,85306,Xanton-Chassenon,1,MAIRIE de XANTON-CHASSENON,NaN,Place de la Mairie,NaN,85240 XANTON-CHASSENON,NaN,85240,...,85-05,-0.694137,46.453547,0.94,street,Place de la Mairie 85240 Xanton-Chassenon,85306_0050,G,PLACE DE LA MAIRIE,NaN
814,85235,Saint-Juire-Champgillon,01 - 1,Bureau de Vote N° 1,NaN,Place de la Mairie,NaN,NaN,NaN,85210,...,85-05,-1.028094,46.579051,0.93,street,Place de la Mairie 85210 Saint-Juire-Champgillon,85235_0250,G,PLACE DE LA MAIRIE,NaN
1498,85248,Saint-Martin-Lars-en-Sainte-Hermine,001,BUREAU N°1 MAIRIE,13,Rue de la mairie,NaN,NaN,NaN,85210,...,85-05,-0.981401,46.590645,0.93,housenumber,13 Rue de la Mairie 85210 Saint-Martin-Lars-en...,85248_0190_00013,G,13 RUE DE LA MAIRIE,NaN
2097,85001,L'Aiguillon-sur-Mer,2,Cantine Scolaire,NaN,boulevard du Communal,NaN,NaN,NaN,85460,...,85-05,-1.300770,46.332414,0.94,street,Boulevard du Communal 85460 L'Aiguillon-sur-Mer,85001_0160,G,BOULEVARD DU COMMUNAL,NaN
2612,85304,Vouillé-les-Marais,1,Bureau 1,NaN,Place de la mairie,NaN,85450 VOUILLE-LES-MARAIS,NaN,85450,...,85-05,-0.966874,46.383666,0.51,street,Route de la Voie Romaine 85450 Vouillé-les-Marais,85304_0066,G,ROUTE DE LA VOIE ROMAINE,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68608,85216,Sainte-Gemme-la-Plaine,002,SALLE DES FETES 2,14,Rue de la Mairie,NaN,NaN,NaN,85400,...,85-05,-1.114964,46.481716,0.94,housenumber,14 Rue de la Mairie 85400 Sainte-Gemme-la-Plaine,85216_0156_00014,G,14 RUE DE LA MAIRIE,NaN
69007,85162,Rives-d'Autise,1,Salle des Fêtes,NaN,rue de la Croix Perrine,Nieul-sur-l'Autise,85240 RIVES-D'AUTISE,NaN,85240,...,85-05,-0.679010,46.425144,0.94,street,Rue de la Croix Perrine 85240 Rives-d'Autise,85162_0081,G,RUE DE LA CROIX PERRINE,NaN
69039,85162,Rives-d'Autise,2,Mairie annexe d'Oulmes,7,rue de la Venise Verte,OULMES,85420 RIVES-D'AUTISE,NaN,85240,...,85-05,-0.663084,46.398917,0.94,housenumber,7 Rue de la Venise Verte 85420 Rives-d'Autise,85168_0069_00007,G,7 RUE DE LA VENISE VERTE,NaN
69332,85136,Marillet,001,MAIRIE,14,Rue des Ajoncs,NaN,NaN,NaN,85240,...,85-05,-0.633484,46.568662,0.93,housenumber,14 Rue des Ajoncs 85240 Marillet,85136_0001_00014,G,14 RUE DES AJONCS,NaN


# Election présidentielle 2017

,Code du département,Département,Code de la circonscription,Circonscription,Code de la commune,Commune,Bureau de vote,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,N°Panneau,Sexe,Nom,Prénom,Voix,% Voix/Ins,% Voix/Exp,Code Insee,Coordonnées,Nom Bureau Vote,Adresse,Code Postal,Ville,uniq_bdv
724,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,5,F,ARTHAUD,Nathalie,2,0.23,0.28,84002,"43.737849,5.464846",Mairie - salle du conseil municipal,Place Saint Elzéar,84240.0,Ansouis,Mairie - salle du conseil municipal - Ansouis
725,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,2,F,LE PEN,Marine,155,17.63,21.92,84002,"43.737849,5.464846",Mairie - salle du conseil municipal,Place Saint Elzéar,84240.0,Ansouis,Mairie - salle du conseil municipal - Ansouis
3,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,8,M,LASSALLE,Jean,9,1.02,1.27,84002,"43.737849,5.464846",Mairie - salle du conseil municipal,Place Saint Elzéar,84240.0,Ansouis,Mairie - salle du conseil municipal - Ansouis
928,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,7,M,CHEMINADE,Jacques,3,0.34,0.42,84002,"43.737849,5.464846",Mairie - salle du conseil municipal,Place Saint Elzéar,84240.0,Ansouis,Mairie - salle du conseil municipal - Ansouis
381,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,3,M,MACRON,Emmanuel,138,15.70,19.52,84002,"43.737849,5.464846",Mairie - salle du conseil municipal,Place Saint Elzéar,84240.0,Ansouis,Mairie - salle du conseil municipal - Ansouis


In [87]:
bureau_pres_2017 = presidentielle_2017.iloc[:, 0:len(cols_bureau + cols_bureau_stats)].drop_duplicates()
bureau_pres_2017.columns = cols_bureau + cols_bureau_stats
bureau_pres_2017.head()

,code_dpt,dpt,code_circ,circ,code_commune,commune,code_b_vote,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot
0,84,Vaucluse,5,5ème circonscription,47,Gargas,1,1278,262,20.50,1016,79.50,19,1.49,1.87,9,0.70,0.89,988,77.31,97.24
1,84,Vaucluse,5,5ème circonscription,31,Carpentras,2,848,232,27.36,616,72.64,14,1.65,2.27,4,0.47,0.65,598,70.52,97.08
2,84,Vaucluse,5,5ème circonscription,4,Aubignan,2,969,214,22.08,755,77.92,7,0.72,0.93,5,0.52,0.66,743,76.68,98.41
3,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93
4,84,Vaucluse,5,5ème circonscription,122,Sarrians,4,909,218,23.98,691,76.02,10,1.10,1.45,5,0.55,0.72,676,74.37,97.83


## Data.gouv.fr

In [96]:
a = {'a':1, 'b': 2}
print(a.keys())

dict_keys(['a', 'b'])


/var/folders/cl/m20zwsz15vb_c47wccs1ghz40000gn/T/ipykernel_41907/1293736181.py:23: DtypeWarning: Columns (0,6) have mixed types. Specify dtype option on import or set low_memory=False.
  pres_2017 = pd.read_csv('./presidentielle_2017_bureau.csv')


,code_dpt,dpt,code_circ,circ,code_commune,commune,code_b_vote,cp,id_b_vote,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,n_panneau,sexe,nom,prenom,voix,p_voix/ins,p_voix/exp
0,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,84002,Ansouis-1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,-,-,abstention,-,142,16.15,0.00
1,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,84002,Ansouis-1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,-,-,blancs,-,24,2.73,3.26
2,84,Vaucluse,5,5ème circonscription,2,Ansouis,1,84002,Ansouis-1,879,142,16.15,737,83.85,24,2.73,3.26,6,0.68,0.81,707,80.43,95.93,-,-,nuls,-,6,0.68,0.81
0,84,Vaucluse,5,5ème circonscription,3,Apt,1,84003,Apt-1,923,189,20.48,734,79.52,10,1.08,1.36,7,0.76,0.95,717,77.68,97.68,-,-,abstention,-,189,20.48,0.00
1,84,Vaucluse,5,5ème circonscription,3,Apt,1,84003,Apt-1,923,189,20.48,734,79.52,10,1.08,1.36,7,0.76,0.95,717,77.68,97.68,-,-,blancs,-,10,1.08,1.36


In [69]:
bureau_pres_2017.abstentions.sum() / bureau_pres_2017.inscrits.sum()

0.1910190615835777

In [84]:
inscrits = bureau_pres_2017.inscrits.sum()
abstentions = bureau_pres_2017.abstentions.sum()
abstentions_p = bureau_pres_2017.abstentions.sum() / bureau_pres_2017.inscrits.sum()
score_melenchon = presidentielle_2017[presidentielle_2017.Nom == "MÉLENCHON"].Voix.sum()
score_melenchon_p = score_melenchon / inscrits
score_macron = presidentielle_2017[presidentielle_2017.Nom == "MACRON"].Voix.sum()
score_macron_p = score_macron / inscrits
score_fillon = presidentielle_2017[presidentielle_2017.Nom == "FILLON"].Voix.sum()
score_fillon_p = score_fillon / inscrits
score_le_pen = presidentielle_2017[presidentielle_2017.Nom == "LE PEN"].Voix.sum()
score_le_pen_p = score_le_pen / inscrits

print('%s inscrits, %s abstention' % (inscrits, abstentions))
print('Score MELENCHON: %s - %s ' % (score_melenchon, round(score_melenchon_p*100, 2) ))
print('Score FILLON: %s - %s ' % (score_fillon, round(score_fillon_p*100, 2) ) )
print('Score MACRON: %s - %s ' % (score_macron, round(score_macron_p*100, 2) ))
print('Score LE PEN: %s - %s ' % (score_le_pen, round(score_le_pen_p*100, 2) ))



81840 inscrits, 15633 abstention
Score MELENCHON: 12244 - 14.96 
Score FILLON: 12862 - 15.72 
Score MACRON: 12954 - 15.83 
Score LE PEN: 18202 - 22.24 


# Elections legislatives 2017

56 communes 

/var/folders/cl/m20zwsz15vb_c47wccs1ghz40000gn/T/ipykernel_41907/695945510.py:2: DtypeWarning: Columns (0,157,158,159,160,165,166,167,168,173,174,175,176,181,182,183,184,189,190,191,192,197,198,199,200,205,206,207,208,213,214,215,216,221,222,223,224,229,230,231,232) have mixed types. Specify dtype option on import or set low_memory=False.
  legislatives = pd.read_csv('./leg_2017_communes.csv')


,Code du département,Libellé du département,Code de la circonscription,Libellé de la circonscription,Code de la commune,Libellé de la commune,Inscrits,Abstentions,% Abs/Ins,Votants,...,% Voix/Ins.25,% Voix/Exp.25,N°Panneau.26,Sexe.26,Nom.26,Prénom.26,Nuance.26,Voix.26,% Voix/Ins.26,% Voix/Exp.26
0,1,Ain,1,1ère circonscription,16,Arbigny,317,160,50.47,157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Ain,1,1ère circonscription,24,Attignat,2325,1266,54.45,1059,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Ain,1,1ère circonscription,29,Beaupont,429,227,52.91,202,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Ain,1,1ère circonscription,38,Bény,575,290,50.43,285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Ain,1,1ère circonscription,40,Béréziat,346,196,56.65,150,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
leg_circo_5 = legislatives[(legislatives['Code de la circonscription'] == 5) & (legislatives['Code du département'] == '84')]

In [41]:
candidats_leg = ["PIERINI Lucien", 
    "CHOTARD Stéphane",
    "REYNARD Luc", 
    "LECELLIER Cédric", 
    "DAURES Anne",
    "KADLER Marie-Christine", 
    "MOREL Véronique", 
    "VIARD Jean", 
    "AUBERT Julien", 
    "CHALENÇON Christophe", 
    "KRAFT Sandrine", 
    "FOREST Nathalie", 
    "THOMAS DE MALEVILLE Marie"]

leg_circo_5 = leg_circo_5.iloc[:, 0:124]
leg_circo_5


,code_dpt,dpt,code_circ,circ,code_commune,commune,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,PIERINI Lucien_n_panneau,PIERINI Lucien_sexe,PIERINI Lucien_nom,PIERINI Lucien_prenom,PIERINI Lucien_nuance,PIERINI Lucien_voix,PIERINI Lucien_p_voix/ins,PIERINI Lucien_p_voix/exp,CHOTARD Stéphane_n_panneau,CHOTARD Stéphane_sexe,CHOTARD Stéphane_nom,CHOTARD Stéphane_prenom,CHOTARD Stéphane_nuance,CHOTARD Stéphane_voix,CHOTARD Stéphane_p_voix/ins,CHOTARD Stéphane_p_voix/exp,REYNARD Luc_n_panneau,REYNARD Luc_sexe,REYNARD Luc_nom,REYNARD Luc_prenom,REYNARD Luc_nuance,REYNARD Luc_voix,REYNARD Luc_p_voix/ins,REYNARD Luc_p_voix/exp,LECELLIER Cédric_n_panneau,LECELLIER Cédric_sexe,LECELLIER Cédric_nom,LECELLIER Cédric_prenom,LECELLIER Cédric_nuance,LECELLIER Cédric_voix,LECELLIER Cédric_p_voix/ins,LECELLIER Cédric_p_voix/exp,DAURES\tAnne_n_panneau,DAURES\tAnne_sexe,DAURES\tAnne_nom,DAURES\tAnne_prenom,DAURES\tAnne_nuance,DAURES\tAnne_voix,DAURES\tAnne_p_voix/ins,DAURES\tAnne_p_voix/exp,KADLER\tMarie-Christine_n_panneau,KADLER\tMarie-Christine_sexe,KADLER\tMarie-Christine_nom,KADLER\tMarie-Christine_prenom,KADLER\tMarie-Christine_nuance,KADLER\tMarie-Christine_voix,KADLER\tMarie-Christine_p_voix/ins,KADLER\tMarie-Christine_p_voix/exp,MOREL Véronique_n_panneau,MOREL Véronique_sexe,MOREL Véronique_nom,MOREL Véronique_prenom,MOREL Véronique_nuance,MOREL Véronique_voix,MOREL Véronique_p_voix/ins,MOREL Véronique_p_voix/exp,VIARD Jean_n_panneau,VIARD Jean_sexe,VIARD Jean_nom,VIARD Jean_prenom,VIARD Jean_nuance,VIARD Jean_voix,VIARD Jean_p_voix/ins,VIARD Jean_p_voix/exp,AUBERT Julien_n_panneau,AUBERT Julien_sexe,AUBERT Julien_nom,AUBERT Julien_prenom,AUBERT Julien_nuance,AUBERT Julien_voix,AUBERT Julien_p_voix/ins,AUBERT Julien_p_voix/exp,CHALENÇON Christophe_n_panneau,CHALENÇON Christophe_sexe,CHALENÇON Christophe_nom,CHALENÇON Christophe_prenom,CHALENÇON Christophe_nuance,CHALENÇON Christophe_voix,CHALENÇON Christophe_p_voix/ins,CHALENÇON Christophe_p_voix/exp,KRAFT Sandrine_n_panneau,KRAFT Sandrine_sexe,KRAFT Sandrine_nom,KRAFT Sandrine_prenom,KRAFT Sandrine_nuance,KRAFT Sandrine_voix,KRAFT Sandrine_p_voix/ins,KRAFT Sandrine_p_voix/exp,FOREST\tNathalie_n_panneau,FOREST\tNathalie_sexe,FOREST\tNathalie_nom,FOREST\tNathalie_prenom,FOREST\tNathalie_nuance,FOREST\tNathalie_voix,FOREST\tNathalie_p_voix/ins,FOREST\tNathalie_p_voix/exp,THOMAS DE MALEVILLE Marie_n_panneau,THOMAS DE MALEVILLE Marie_sexe,THOMAS DE MALEVILLE Marie_nom,THOMAS DE MALEVILLE Marie_prenom,THOMAS DE MALEVILLE Marie_nuance,THOMAS DE MALEVILLE Marie_voix,THOMAS DE MALEVILLE Marie_p_voix/ins,THOMAS DE MALEVILLE Marie_p_voix/exp
33094,84,Vaucluse,5,5ème circonscription,2,Ansouis,880,419,47.61,461,52.39,1,0.11,0.22,14,1.59,3.04,446,50.68,96.75,6,M,PIERINI,Lucien,EXG,3,0.34,0.67,8,M,CHOTARD,Stéphane,FI,66,7.50,14.80,7,M,REYNARD,Luc,RDG,4,0.45,0.90,5.0,M,LECELLIER,Cédric,DVG,3.0,0.34,0.67,9.0,F,DAURES,Anne,ECO,10.0,1.14,2.24,10.0,F,KADLER,Marie-Christine,ECO,11.0,1.25,2.47,13.0,F,MOREL,Véronique,DIV,5.0,0.57,1.12,11.0,M,VIARD,Jean,REM,172.0,19.55,38.57,2.0,M,AUBERT,Julien,LR,95.0,10.80,21.30,1.0,M,CHALENÇON,Christophe,DVD,1.0,0.11,0.22,4.0,F,KRAFT,Sandrine,DVD,3.0,0.34,0.67,12.0,F,FOREST,Nathalie,DLF,13.0,1.48,2.91,3.0,F,THOMAS DE MALEVILLE,Marie,FN,60.0,6.82,13.45
33095,84,Vaucluse,5,5ème circonscription,3,Apt,7401,4088,55.24,3313,44.76,56,0.76,1.69,21,0.28,0.63,3236,43.72,97.68,6,M,PIERINI,Lucien,EXG,31,0.42,0.96,8,M,CHOTARD,Stéphane,FI,419,5.66,12.95,7,M,REYNARD,Luc,RDG,48,0.65,1.48,5.0,M,LECELLIER,Cédric,DVG,20.0,0.27,0.62,9.0,F,DAURES,Anne,ECO,45.0,0.61,1.39,10.0,F,KADLER,Marie-Christine,ECO,150.0,2.03,4.64,13.0,F,MOREL,Véronique,DIV,23.0,0.31,0.71,11.0,M,VIARD,Jean,REM,1041.0,14.07,32.17,2.0,M,AUBERT,Julien,LR,874.0,11.81,27.01,1.0,M,CHALENÇON,Christophe,DVD,10.0,0.14,0.31,4.0,F,KRAFT,Sandrine,DVD,63.0,0.85,1.95,12.0,F,FOREST,Nathalie,DLF,44.0,0.59,1.36,3.0,F,THOMAS DE MALEVILLE,Marie,FN,4

In [43]:
from itertools import product
all_candidates_columns = list(map(lambda x: x[0] + '_' + x[1], product(candidats_leg, cols_candidates_leg)))
all_candidates_columns

['PIERINI Lucien_n_panneau',
 'PIERINI Lucien_sexe',
 'PIERINI Lucien_nom',
 'PIERINI Lucien_prenom',
 'PIERINI Lucien_nuance',
 'PIERINI Lucien_voix',
 'PIERINI Lucien_p_voix/ins',
 'PIERINI Lucien_p_voix/exp',
 'CHOTARD Stéphane_n_panneau',
 'CHOTARD Stéphane_sexe',
 'CHOTARD Stéphane_nom',
 'CHOTARD Stéphane_prenom',
 'CHOTARD Stéphane_nuance',
 'CHOTARD Stéphane_voix',
 'CHOTARD Stéphane_p_voix/ins',
 'CHOTARD Stéphane_p_voix/exp',
 'REYNARD Luc_n_panneau',
 'REYNARD Luc_sexe',
 'REYNARD Luc_nom',
 'REYNARD Luc_prenom',
 'REYNARD Luc_nuance',
 'REYNARD Luc_voix',
 'REYNARD Luc_p_voix/ins',
 'REYNARD Luc_p_voix/exp',
 'LECELLIER Cédric_n_panneau',
 'LECELLIER Cédric_sexe',
 'LECELLIER Cédric_nom',
 'LECELLIER Cédric_prenom',
 'LECELLIER Cédric_nuance',
 'LECELLIER Cédric_voix',
 'LECELLIER Cédric_p_voix/ins',
 'LECELLIER Cédric_p_voix/exp',
 'DAURES Anne_n_panneau',
 'DAURES Anne_sexe',
 'DAURES Anne_nom',
 'DAURES Anne_prenom',
 'DAURES Anne_nuance',
 'DAURES Anne_voix',
 'DAURES A

In [44]:

leg_circo_5.columns = cols_bureau_legs + cols_bureau_stats + all_candidates_columns
leg_circo_5

,code_dpt,dpt,code_circ,circ,code_commune,commune,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,PIERINI Lucien_n_panneau,PIERINI Lucien_sexe,PIERINI Lucien_nom,PIERINI Lucien_prenom,PIERINI Lucien_nuance,PIERINI Lucien_voix,PIERINI Lucien_p_voix/ins,PIERINI Lucien_p_voix/exp,CHOTARD Stéphane_n_panneau,CHOTARD Stéphane_sexe,CHOTARD Stéphane_nom,CHOTARD Stéphane_prenom,CHOTARD Stéphane_nuance,CHOTARD Stéphane_voix,CHOTARD Stéphane_p_voix/ins,CHOTARD Stéphane_p_voix/exp,REYNARD Luc_n_panneau,REYNARD Luc_sexe,REYNARD Luc_nom,REYNARD Luc_prenom,REYNARD Luc_nuance,REYNARD Luc_voix,REYNARD Luc_p_voix/ins,REYNARD Luc_p_voix/exp,LECELLIER Cédric_n_panneau,LECELLIER Cédric_sexe,LECELLIER Cédric_nom,LECELLIER Cédric_prenom,LECELLIER Cédric_nuance,LECELLIER Cédric_voix,LECELLIER Cédric_p_voix/ins,LECELLIER Cédric_p_voix/exp,DAURES Anne_n_panneau,DAURES Anne_sexe,DAURES Anne_nom,DAURES Anne_prenom,DAURES Anne_nuance,DAURES Anne_voix,DAURES Anne_p_voix/ins,DAURES Anne_p_voix/exp,KADLER Marie-Christine_n_panneau,KADLER Marie-Christine_sexe,KADLER Marie-Christine_nom,KADLER Marie-Christine_prenom,KADLER Marie-Christine_nuance,KADLER Marie-Christine_voix,KADLER Marie-Christine_p_voix/ins,KADLER Marie-Christine_p_voix/exp,MOREL Véronique_n_panneau,MOREL Véronique_sexe,MOREL Véronique_nom,MOREL Véronique_prenom,MOREL Véronique_nuance,MOREL Véronique_voix,MOREL Véronique_p_voix/ins,MOREL Véronique_p_voix/exp,VIARD Jean_n_panneau,VIARD Jean_sexe,VIARD Jean_nom,VIARD Jean_prenom,VIARD Jean_nuance,VIARD Jean_voix,VIARD Jean_p_voix/ins,VIARD Jean_p_voix/exp,AUBERT Julien_n_panneau,AUBERT Julien_sexe,AUBERT Julien_nom,AUBERT Julien_prenom,AUBERT Julien_nuance,AUBERT Julien_voix,AUBERT Julien_p_voix/ins,AUBERT Julien_p_voix/exp,CHALENÇON Christophe_n_panneau,CHALENÇON Christophe_sexe,CHALENÇON Christophe_nom,CHALENÇON Christophe_prenom,CHALENÇON Christophe_nuance,CHALENÇON Christophe_voix,CHALENÇON Christophe_p_voix/ins,CHALENÇON Christophe_p_voix/exp,KRAFT Sandrine_n_panneau,KRAFT Sandrine_sexe,KRAFT Sandrine_nom,KRAFT Sandrine_prenom,KRAFT Sandrine_nuance,KRAFT Sandrine_voix,KRAFT Sandrine_p_voix/ins,KRAFT Sandrine_p_voix/exp,FOREST Nathalie_n_panneau,FOREST Nathalie_sexe,FOREST Nathalie_nom,FOREST Nathalie_prenom,FOREST Nathalie_nuance,FOREST Nathalie_voix,FOREST Nathalie_p_voix/ins,FOREST Nathalie_p_voix/exp,THOMAS DE MALEVILLE Marie_n_panneau,THOMAS DE MALEVILLE Marie_sexe,THOMAS DE MALEVILLE Marie_nom,THOMAS DE MALEVILLE Marie_prenom,THOMAS DE MALEVILLE Marie_nuance,THOMAS DE MALEVILLE Marie_voix,THOMAS DE MALEVILLE Marie_p_voix/ins,THOMAS DE MALEVILLE Marie_p_voix/exp
33094,84,Vaucluse,5,5ème circonscription,2,Ansouis,880,419,47.61,461,52.39,1,0.11,0.22,14,1.59,3.04,446,50.68,96.75,6,M,PIERINI,Lucien,EXG,3,0.34,0.67,8,M,CHOTARD,Stéphane,FI,66,7.50,14.80,7,M,REYNARD,Luc,RDG,4,0.45,0.90,5.0,M,LECELLIER,Cédric,DVG,3.0,0.34,0.67,9.0,F,DAURES,Anne,ECO,10.0,1.14,2.24,10.0,F,KADLER,Marie-Christine,ECO,11.0,1.25,2.47,13.0,F,MOREL,Véronique,DIV,5.0,0.57,1.12,11.0,M,VIARD,Jean,REM,172.0,19.55,38.57,2.0,M,AUBERT,Julien,LR,95.0,10.80,21.30,1.0,M,CHALENÇON,Christophe,DVD,1.0,0.11,0.22,4.0,F,KRAFT,Sandrine,DVD,3.0,0.34,0.67,12.0,F,FOREST,Nathalie,DLF,13.0,1.48,2.91,3.0,F,THOMAS DE MALEVILLE,Marie,FN,60.0,6.82,13.45
33095,84,Vaucluse,5,5ème circonscription,3,Apt,7401,4088,55.24,3313,44.76,56,0.76,1.69,21,0.28,0.63,3236,43.72,97.68,6,M,PIERINI,Lucien,EXG,31,0.42,0.96,8,M,CHOTARD,Stéphane,FI,419,5.66,12.95,7,M,REYNARD,Luc,RDG,48,0.65,1.48,5.0,M,LECELLIER,Cédric,DVG,20.0,0.27,0.62,9.0,F,DAURES,Anne,ECO,45.0,0.61,1.39,10.0,F,KADLER,Marie-Christine,ECO,150.0,2.03,4.64,13.0,F,MOREL,Véronique,DIV,23.0,0.31,0.71,11.0,M,VIARD,Jean,REM,1041.0,14.07,32.17,2.0,M,AUBERT,Julien,LR,874.0,11.81,27.01,1.0,M,CHALENÇON,Christophe,DVD,10.0,0.14,0.31,4.0,F,KRAFT,Sandrine,DVD,63.0,0.85,1.95,12.0,F,FOREST,Nathalie,DLF,44.0,0.59,1.36,3.0,F,THOMAS DE MALEVILLE,Marie,FN,468.0,6.32,14.46
33096,84

In [45]:
leg_circo_5['cp'] = leg_circo_5.code_dpt.astype(str) + list(map(lambda s: str(s).rjust(3, '0'), leg_circo_5.code_commune))

leg_circo_5 = leg_circo_5[ cols_bureau_legs + ['cp'] + cols_bureau_stats  + list(map(lambda x: x[0] + '_' + x[1], product(candidats_leg, cols_candidates_leg)))  ]
leg_circo_5.head()

,code_dpt,dpt,code_circ,circ,code_commune,commune,cp,inscrits,abstentions,p_abs/ins,votants,p_vot/ins,blancs,p_blancs/ins,p_blancs/vot,nuls,p_nuls/ins,p_nuls/vot,exprimés,p_exp/ins,p_exp/vot,PIERINI Lucien_n_panneau,PIERINI Lucien_sexe,PIERINI Lucien_nom,PIERINI Lucien_prenom,PIERINI Lucien_nuance,PIERINI Lucien_voix,PIERINI Lucien_p_voix/ins,PIERINI Lucien_p_voix/exp,CHOTARD Stéphane_n_panneau,CHOTARD Stéphane_sexe,CHOTARD Stéphane_nom,CHOTARD Stéphane_prenom,CHOTARD Stéphane_nuance,CHOTARD Stéphane_voix,CHOTARD Stéphane_p_voix/ins,CHOTARD Stéphane_p_voix/exp,REYNARD Luc_n_panneau,REYNARD Luc_sexe,REYNARD Luc_nom,REYNARD Luc_prenom,REYNARD Luc_nuance,REYNARD Luc_voix,REYNARD Luc_p_voix/ins,REYNARD Luc_p_voix/exp,LECELLIER Cédric_n_panneau,LECELLIER Cédric_sexe,LECELLIER Cédric_nom,LECELLIER Cédric_prenom,LECELLIER Cédric_nuance,LECELLIER Cédric_voix,LECELLIER Cédric_p_voix/ins,LECELLIER Cédric_p_voix/exp,DAURES Anne_n_panneau,DAURES Anne_sexe,DAURES Anne_nom,DAURES Anne_prenom,DAURES Anne_nuance,DAURES Anne_voix,DAURES Anne_p_voix/ins,DAURES Anne_p_voix/exp,KADLER Marie-Christine_n_panneau,KADLER Marie-Christine_sexe,KADLER Marie-Christine_nom,KADLER Marie-Christine_prenom,KADLER Marie-Christine_nuance,KADLER Marie-Christine_voix,KADLER Marie-Christine_p_voix/ins,KADLER Marie-Christine_p_voix/exp,MOREL Véronique_n_panneau,MOREL Véronique_sexe,MOREL Véronique_nom,MOREL Véronique_prenom,MOREL Véronique_nuance,MOREL Véronique_voix,MOREL Véronique_p_voix/ins,MOREL Véronique_p_voix/exp,VIARD Jean_n_panneau,VIARD Jean_sexe,VIARD Jean_nom,VIARD Jean_prenom,VIARD Jean_nuance,VIARD Jean_voix,VIARD Jean_p_voix/ins,VIARD Jean_p_voix/exp,AUBERT Julien_n_panneau,AUBERT Julien_sexe,AUBERT Julien_nom,AUBERT Julien_prenom,AUBERT Julien_nuance,AUBERT Julien_voix,AUBERT Julien_p_voix/ins,AUBERT Julien_p_voix/exp,CHALENÇON Christophe_n_panneau,CHALENÇON Christophe_sexe,CHALENÇON Christophe_nom,CHALENÇON Christophe_prenom,CHALENÇON Christophe_nuance,CHALENÇON Christophe_voix,CHALENÇON Christophe_p_voix/ins,CHALENÇON Christophe_p_voix/exp,KRAFT Sandrine_n_panneau,KRAFT Sandrine_sexe,KRAFT Sandrine_nom,KRAFT Sandrine_prenom,KRAFT Sandrine_nuance,KRAFT Sandrine_voix,KRAFT Sandrine_p_voix/ins,KRAFT Sandrine_p_voix/exp,FOREST Nathalie_n_panneau,FOREST Nathalie_sexe,FOREST Nathalie_nom,FOREST Nathalie_prenom,FOREST Nathalie_nuance,FOREST Nathalie_voix,FOREST Nathalie_p_voix/ins,FOREST Nathalie_p_voix/exp,THOMAS DE MALEVILLE Marie_n_panneau,THOMAS DE MALEVILLE Marie_sexe,THOMAS DE MALEVILLE Marie_nom,THOMAS DE MALEVILLE Marie_prenom,THOMAS DE MALEVILLE Marie_nuance,THOMAS DE MALEVILLE Marie_voix,THOMAS DE MALEVILLE Marie_p_voix/ins,THOMAS DE MALEVILLE Marie_p_voix/exp
33094,84,Vaucluse,5,5ème circonscription,2,Ansouis,84002,880,419,47.61,461,52.39,1,0.11,0.22,14,1.59,3.04,446,50.68,96.75,6,M,PIERINI,Lucien,EXG,3,0.34,0.67,8,M,CHOTARD,Stéphane,FI,66,7.50,14.80,7,M,REYNARD,Luc,RDG,4,0.45,0.90,5.0,M,LECELLIER,Cédric,DVG,3.0,0.34,0.67,9.0,F,DAURES,Anne,ECO,10.0,1.14,2.24,10.0,F,KADLER,Marie-Christine,ECO,11.0,1.25,2.47,13.0,F,MOREL,Véronique,DIV,5.0,0.57,1.12,11.0,M,VIARD,Jean,REM,172.0,19.55,38.57,2.0,M,AUBERT,Julien,LR,95.0,10.80,21.30,1.0,M,CHALENÇON,Christophe,DVD,1.0,0.11,0.22,4.0,F,KRAFT,Sandrine,DVD,3.0,0.34,0.67,12.0,F,FOREST,Nathalie,DLF,13.0,1.48,2.91,3.0,F,THOMAS DE MALEVILLE,Marie,FN,60.0,6.82,13.45
33095,84,Vaucluse,5,5ème circonscription,3,Apt,84003,7401,4088,55.24,3313,44.76,56,0.76,1.69,21,0.28,0.63,3236,43.72,97.68,6,M,PIERINI,Lucien,EXG,31,0.42,0.96,8,M,CHOTARD,Stéphane,FI,419,5.66,12.95,7,M,REYNARD,Luc,RDG,48,0.65,1.48,5.0,M,LECELLIER,Cédric,DVG,20.0,0.27,0.62,9.0,F,DAURES,Anne,ECO,45.0,0.61,1.39,10.0,F,KADLER,Marie-Christine,ECO,150.0,2.03,4.64,13.0,F,MOREL,Véronique,DIV,23.0,0.31,0.71,11.0,M,VIARD,Jean,REM,1041.0,14.07,32.17,2.0,M,AUBERT,Julien,LR,874.0,11.81,27.01,1.0,M,CHALENÇON,Christophe,DVD,10.0,0.14,0.31,4.0,F,KRAFT,Sandrine,DVD,63.0,0.85,1.95,12.0,F,FOREST,Nathalie,DLF,44.0,0.59,1.36,3.0,F,THOMAS DE MALEVILLE,Marie,FN,468.0,6.32